In [ ]:
!pip install langdetect

from pyspark.sql import SparkSession
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from langdetect import detect, DetectorFactory

import json
import numpy as np
import torch
import pandas as pd

In [ ]:

# Set seed for reproducibility
DetectorFactory.seed = 0

# Load the JSON data
with open('bus_reviews.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)
df = df[['Bus Name', 'Comment']].rename(columns={'Bus Name': 'bus_name', 'Comment': 'comment'})

# Function to detect language
def detect_language(comment):
    try:
        return detect(comment)
    except:
        return None

# Split data into English and Vietnamese comments
df_vi = df[df['comment'].apply(lambda x: detect_language(x) == 'vi')].reset_index(drop=True)
df_en = df[df['comment'].apply(lambda x: detect_language(x) == 'en')].reset_index(drop=True)

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Giả sử 'bus_names' và 'sentences' là danh sách các tên nhà xe và bình luận
bus_names = df_en['bus_name']
sentences = df_en['comment']

# Load tokenizer và model
model_path = '5CD-AI/Vietnamese-Sentiment-visobert'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Di chuyển model vào GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Định nghĩa batch size
batch_size = 16  # Điều chỉnh dựa trên bộ nhớ GPU

# Danh sách để lưu kết quả
results = []

# Xử lý câu theo batch
for start_idx in range(0, len(sentences), batch_size):
    batch_bus_names = bus_names[start_idx:start_idx + batch_size].tolist()
    batch_sentences = sentences[start_idx:start_idx + batch_size].tolist()

    # Encode sentences
    inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Dự đoán
    with torch.no_grad():
        outputs = model(**inputs)
        scores = outputs.logits.softmax(dim=-1).cpu().numpy()

    # Lưu kết quả vào danh sách
    for i, (bus_name, sentence) in enumerate(zip(batch_bus_names, batch_sentences)):
        results.append({
            'Bus_Name': bus_name,
            'Comment': sentence,
            'POS': round(scores[i][model.config.label2id['POSITIVE']], 4),
            'NEG': round(scores[i][model.config.label2id['NEGATIVE']], 4)
            'NEU': round(scores[i][model.config.label2id['NEUTRAL']], 4)
        })

    # Giải phóng bộ nhớ GPU
    torch.cuda.empty_cache()

# Chuyển danh sách kết quả thành DataFrame
result_df = pd.DataFrame(results)

# Lưu vào file CSV
result_df.to_csv("sentiment_vi_results.csv", index=False, encoding="utf-8-sig")

print("Đã lưu kết quả vào sentiment_vi_results.csv")